In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import pickle
sns.set_theme()

# jupyter notebook full-width display
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# no text wrapping
display(HTML("<style>.dataframe td { white-space: nowrap; }</style>"))

# pandas formatting
pd.set_option('display.float_format', '{:.3f}'.format)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 600)
pd.set_option('display.max_colwidth', 200)

In [2]:
# base dataframes
df_hist = pd.read_pickle('calculations\df_hist.pickle')
df_spec = pd.read_pickle('calculations\df_spec.pickle')
df_arch = pd.read_pickle('calculations\df_arch.pickle')
df_prob = pd.read_pickle('calculations\df_prob.pickle')
df_matching_stats = pd.read_pickle('calculations\df_matching_stats.pickle')
df_summary = pd.read_pickle('calculations\df_summary.pickle')

# drop error calc from last notebook - improve in this notebook
df_summary = df_summary.drop(['length_MSE', 'length_ME'], axis=1, level=0)

In [3]:
# import saved lists
with open('calculations\\exact_counts_match.pickle', 'rb') as f:
    exact_counts_match = pickle.load(f)
with open('calculations\\strong_sample_matches_tolerance_0.pickle', 'rb') as f:
    strong_sample_matches_tolerance_0 = pickle.load(f)
with open('calculations\\strong_sample_matches_tolerance_1.pickle', 'rb') as f:
    strong_sample_matches_tolerance_0 = pickle.load(f)
with open('calculations\\strong_sample_matches_tolerance_2.pickle', 'rb') as f:
    strong_sample_matches_tolerance_0 = pickle.load(f)
with open('calculations\\potential_fish_matches_tolerance_0.pickle', 'rb') as f:
    potential_fish_matches_tolerance_0 = pickle.load(f)
with open('calculations\\potential_fish_matches_tolerance_1.pickle', 'rb') as f:
    potential_fish_matches_tolerance_1 = pickle.load(f)
with open('calculations\\potential_fish_matches_tolerance_2.pickle', 'rb') as f:
    potential_fish_matches_tolerance_2 = pickle.load(f)
with open('calculations\\bad_sample_matches_tolerance_0.pickle', 'rb') as f:
    bad_sample_matches_tolerance_0 = pickle.load(f)
with open('calculations\\bad_sample_matches_tolerance_1.pickle', 'rb') as f:
    bad_sample_matches_tolerance_1 = pickle.load(f)
with open('calculations\\bad_sample_matches_tolerance_2.pickle', 'rb') as f:
    bad_sample_matches_tolerance_2 = pickle.load(f)

# NOT DOUBLE COUNTED

In [47]:
# no specimen data == not double counted
[x for x in df_hist.sample_id.unique() if x not in df_summary.index]

[4390,
 4391,
 4392,
 4393,
 4394,
 4395,
 4396,
 4397,
 4398,
 4399,
 4400,
 4408,
 4599,
 5133,
 5319,
 7877,
 7880,
 7884]

In [54]:
# more bio data than spec data == at least some bio samples aren't double counted
df_summary[df_summary.n_hist > df_summary.n_spec].iloc[:, 26:]

,n_matches,n_hist,n_spec,matches_proportion
,,,,
sample_id,,,,
5246,3,22,7,0.136
5266,1,14,2,0.071
5270,1,20,3,0.050
5271,3,30,5,0.100
5348,1,11,10,0.091
5357,2,20,2,0.100
7532,1,17,6,0.059
7857,27,51,32,0.529


# improved bins, plotting and error checking

In [34]:
# bins used by specimen table
# looks like bio data doesn't need to end with a 3 or 8
(df_spec.fork_length % 10).value_counts()

3    38505
8    34192
0      242
9      231
4      228
2      225
5      224
1      207
7      206
6      187
Name: fork_length, dtype: Int64

In [74]:
df_spec.notna().sum()
# note: there are only 6 sex data and they are the import error (should be bio) from the other notebook

id                      74666
fork_length             74447
weight                    429
river_age               74664
notes                   74666
sample_id               74666
sex_id                      6
status_id               74666
age_type                74664
sweep_id                74666
life_stage_id           74666
old_id                  74666
smart_river_age         74664
smart_river_age_type    74664
matching_id             74666
dtype: int64

In [71]:
# check: confirm that values ending in not 3/8 have more weight/sex

# still no sex data, but most of the weight data is here 
# (you would expect exactly 80% if it was 100% because more detailed measurements could include n%5==3)

df_spec[df_spec.fork_length % 5 != 3][['weight', 'sex_id']].notna().sum() / df_spec[['weight', 'sex_id']].notna().sum()

weight   0.841
sex_id   0.000
dtype: float64

In [35]:
# bins go from 23 to 163
df_spec.fork_length.describe()

count   74447.000
mean       57.040
std        23.221
min        23.000
25%        43.000
50%        48.000
75%        68.000
max       163.000
Name: fork_length, dtype: float64

# improved fork length plotting function
using bins as utilised by specimen table

In [42]:
def plot_fork_length_by_sample(df_specimen, df_historical, sample_id, feature='fork_length', bin_width=5, density=False, subtitle=''):
    
    figsize=(16,4)
    bins_plot = [x*5 + 20 for x in range(30)]  # centered on n%5==3 like df_spec, rounded to int%5 (could use +20.5 also)
        
    plt.figure(figsize=figsize)
    plt.xlim(20, 170)  # use same scale for all histograms for easy comparison
    df_specimen.loc[df_specimen.sample_id==sample_id, feature].dropna().hist(alpha=0.5, color=sns.color_palette()[0], density=density, label=f'Specimen {feature}', bins=bins_plot)
    df_historical.loc[df_historical.sample_id==sample_id, feature].dropna().hist(alpha=0.5, color=sns.color_palette()[1], density=density, label=f'Bioligical {feature}', bins=bins_plot)
    
    feature_title = feature.title().replace("_"," ")
    plt.legend(loc='upper right')
    subtitle = ' - ' + subtitle if subtitle else ''
    plt.title(f'Sample {sample_id}: {feature_title} Comparison - Specimen vs Biological Data{subtitle}')
    plt.ylabel('Counts')
    plt.xlabel(f'{feature_title}')
    plt.show()

# improved error calculating
using bins as utilised by the specimen table

In [286]:
%%time

weight_tolerance = 1
potential_fish_matches = []  # list(sample, spec, hist, hist_total) - these should only trigger if an exact match on sex/len/wt within tolerance, if exists
strong_sample_matches = list()  # a match is found for every fish in df_hist - sample likely contains duplicated spec/bio
bad_sample_matches = set()  # df_hist contains unmatchable fish - some fish are definitely not duplicated spec/bio
last_sample = 0
df = pd.DataFrame()
hist_total, hist_matches = 999, 0 

for i, row in df_hist.sort_values(['sample_id', 'id']).iterrows():

    fish_id, sample_id, fork_length, weight, sex_id = row[['id', 'sample_id', 'fork_length', 'weight', 'sex_id']]
    current_bin = fork_length - fork_length%5, fork_length - fork_length%5 + 5  # these are int bins n%5, could add 0.5 per above note
    
    if last_sample != sample_id:
        df = df_spec[df_spec.sample_id==sample_id]
        # strong matches
        if hist_matches == hist_total:
            strong_sample_matches += [last_sample]
        hist_matches = 0
        hist_total = df_hist[df_hist.sample_id==sample_id].shape[0]
        
    if not df.empty:
        
        results = df[
            ((df.fork_length>=current_bin[0]) & (df.fork_length<current_bin[1])) # check if fork_length is in the same bin
            & (
                ((df.weight>=weight-weight_tolerance) & (df.weight>=weight-weight_tolerance))
                | df.weight.isnull()
            )
            & ((df.sex_id==sex_id) | df.sex_id.isnull())
        ]
        if not results.empty:
            hist_matches += 1
            potential_fish_matches += [[sample_id, fish_id, results.iloc[0].id, hist_total, hist_matches, fork_length, results.iloc[[0]].fork_length.values[0]]]
            df = df.drop(results.iloc[[0]].index[0]) # drop this row so it doesn't get matched again
        else:
            bad_sample_matches.add(sample_id)  # triggers if results is empty (there are no matches)

    else:
        bad_sample_matches.add(sample_id)  # triggers if df is empty

    last_sample = sample_id

Wall time: 1min


In [287]:
# use potential fish matches to calculate error
error_penalty_per_unmatched_fish = 100  # arbitrary

df_matches = pd.DataFrame(potential_fish_matches, columns=['sample_id', 'hist_id', 'spec_id', 'total_hist', 'cumulative_matches', 'hist_fork_length', 'spec_fork_length'])
df_match_counts = df_matches.groupby('sample_id').max()[['cumulative_matches', 'total_hist']].rename({'total_hist':'total', 'cumulative_matches':'matches'}, axis=1)

df_matches = pd.merge(
    df_matches,
    df_match_counts,
    on='sample_id',
    how='left'
).drop(['cumulative_matches', 'total_hist'], axis=1)

df_matches['fish_sq_error'] = (df_matches['hist_fork_length'] - df_matches['spec_fork_length']) ** 2
df_matches['unmatched_penalty'] = (df_matches['total'] - df_matches['matches']) * error_penalty_per_unmatched_fish
df_matches = df_matches.merge(
    pd.DataFrame(df_matches[['sample_id', 'fish_sq_error', 'unmatched_penalty']].groupby('sample_id').agg({'fish_sq_error':'sum', 'unmatched_penalty':'max'}).sum(axis=1), columns=['sample_SSE']),
    on='sample_id',
    how='left'
).drop(['fish_sq_error', 'unmatched_penalty'], axis=1)

df_matches.groupby('sample_id').max()[['matches', 'total', 'sample_SSE']].sort_values('sample_SSE')

,matches,total,sample_SSE
sample_id,,,
7611,1,1,0
4485,1,1,0
4578,5,5,0
7639,1,1,1
7691,1,1,1
...,...,...,...
7644,138,174,4251
7653,231,266,4485
7637,225,270,5472


In [288]:
%%time
# if we sort descending, do we get the same number of strong and impossible matches?

weight_tolerance = 1
potential_fish_matches = []  # list(sample, spec, hist, hist_total) - these should only trigger if an exact match on sex/len/wt within tolerance, if exists
strong_sample_matches = list()  # a match is found for every fish in df_hist - sample likely contains duplicated spec/bio
bad_sample_matches = set()  # df_hist contains unmatchable fish - some fish are definitely not duplicated spec/bio
last_sample = 0
df = pd.DataFrame()
hist_total, hist_matches = 999, 0 

for i, row in df_hist.sort_values(['sample_id', 'id'], ascending=False).iterrows():

    fish_id, sample_id, fork_length, weight, sex_id = row[['id', 'sample_id', 'fork_length', 'weight', 'sex_id']]
    current_bin = fork_length - fork_length%5, fork_length - fork_length%5 + 5  # these are int bins n%5, could add 0.5 per above note
    
    if last_sample != sample_id:
        df = df_spec[df_spec.sample_id==sample_id]
        # strong matches
        if hist_matches == hist_total:
            strong_sample_matches += [last_sample]
        hist_matches = 0
        hist_total = df_hist[df_hist.sample_id==sample_id].shape[0]
        
    if not df.empty:
        
        results = df[
            ((df.fork_length>=current_bin[0]) & (df.fork_length<current_bin[1])) # check if fork_length is in the same bin
            & (
                ((df.weight>=weight-weight_tolerance) & (df.weight>=weight-weight_tolerance))
                | df.weight.isnull()
            )
            & ((df.sex_id==sex_id) | df.sex_id.isnull())
        ]
        if not results.empty:
            hist_matches += 1
            potential_fish_matches += [[sample_id, fish_id, results.iloc[0].id, hist_total, hist_matches, fork_length, results.iloc[[0]].fork_length.values[0]]]
            df = df.drop(results.iloc[[0]].index[0]) # drop this row so it doesn't get matched again
        else:
            bad_sample_matches.add(sample_id)  # triggers if results is empty (there are no matches)

    else:
        bad_sample_matches.add(sample_id)  # triggers if df is empty

    last_sample = sample_id
    
    
# use potential fish matches to calculate error
error_penalty_per_unmatched_fish = 100  # arbitrary

df_matches_desc = pd.DataFrame(potential_fish_matches, columns=['sample_id', 'hist_id', 'spec_id', 'total_hist', 'cumulative_matches', 'hist_fork_length', 'spec_fork_length'])
df_match_counts = df_matches_desc.groupby('sample_id').max()[['cumulative_matches', 'total_hist']].rename({'total_hist':'total', 'cumulative_matches':'matches'}, axis=1)

df_matches_desc = pd.merge(
    df_matches_desc,
    df_match_counts,
    on='sample_id',
    how='left'
).drop(['cumulative_matches', 'total_hist'], axis=1)

df_matches_desc['fish_sq_error'] = (df_matches_desc['hist_fork_length'] - df_matches_desc['spec_fork_length']) ** 2
df_matches_desc['unmatched_penalty'] = (df_matches_desc['total'] - df_matches_desc['matches']) * error_penalty_per_unmatched_fish
df_matches_desc = df_matches_desc.merge(
    pd.DataFrame(df_matches_desc[['sample_id', 'fish_sq_error', 'unmatched_penalty']].groupby('sample_id').agg({'fish_sq_error':'sum', 'unmatched_penalty':'max'}).sum(axis=1), columns=['sample_SSE']),
    on='sample_id',
    how='left'
).drop(['fish_sq_error', 'unmatched_penalty'], axis=1)

df_matches_desc.groupby('sample_id').max()[['matches', 'total', 'sample_SSE']].sort_values('sample_SSE')

Wall time: 59.8 s


,matches,total,sample_SSE
sample_id,,,
4578,5,5,0
7611,1,1,0
4485,1,1,0
4635,2,2,1
4634,1,1,1
...,...,...,...
7622,238,271,4071
7653,231,266,4241
7637,225,270,5136


            matches	total	sample_SSE
    sample_id			
    7611	1	1	0
    4485	1	1	0
    4578	5	5	0
    7639	1	1	1
    7691	1	1	1
    ...	...	...	...
    7644	138	174	4251
    7653	231	266	4485
    7637	225	270	5472
    7635	223	275	6095
    7591	290	346	6865

In [312]:
SSE_comparison = pd.merge(
    df_matches.groupby('sample_id').max()['sample_SSE'].reset_index().rename({'sample_SSE':'SSE_asc'}, axis=1),
    df_matches_desc.groupby('sample_id').max()['sample_SSE'].reset_index().rename({'sample_SSE':'SSE_desc'}, axis=1),
    on='sample_id'
)
SSE_comparison['delta'] = SSE_comparison['SSE_asc'] - SSE_comparison['SSE_desc']
SSE_comparison['delta_scaled'] = (SSE_comparison['delta'] / ((SSE_comparison['SSE_asc'] + SSE_comparison['SSE_desc']) / 2)).abs().fillna(0)
SSE_comparison.sort_values('delta_scaled', ascending=False).head(22)
SSE_comparison.describe(percentiles=[0.95, 0.975,0.995])

# these look different enough that we should run both and take the best match

,sample_id,SSE_asc,SSE_desc,delta,delta_scaled
count,768.000,768.000,768.000,768.000,768.000
mean,6406.895,535.060,527.673,7.387,0.061
std,1345.255,773.377,738.046,67.110,0.200
min,4404.000,0.000,0.000,-614.000,0.000
50%,7169.500,274.000,302.500,4.000,0.016
95%,7862.650,1879.300,1789.700,81.000,0.190
97.5%,7928.825,2988.475,2824.775,122.775,0.769
99.5%,7990.165,4289.610,4099.050,232.310,1.429
max,8001.000,6865.000,6548.000,336.000,1.646
